In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create a SparkSession
spark = SparkSession.builder.appName("App").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

24/01/04 23:14:32 WARN Utils: Your hostname, andreas-HP-Pavilion-Laptop-15-cs1xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.61 instead (on interface wlo1)
24/01/04 23:14:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 23:14:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/04 23:14:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Load CSV File with Header

In [2]:
# Load the CSV file with the first row as a header
df = spark.read.format("csv").option("delimiter", ",").option("header", "true").load("1987.csv")

# Display the columns and the first 15 rows
df.show(15, truncate=False)

24/01/04 23:15:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1987|10   |14        |3        |741    |730       |912    |849       |PS           |1451     

In [3]:
# Iterate over all columns in the DataFrame
for column in df.columns:
    df = df.withColumn(column, when(col(column) == "NA", None).otherwise(col(column)))

df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1987|   10|        14|        3|    741|       730|    912|       849|           PS|     1451

In [4]:
from pyspark.sql.functions import col

# Counts the number of null values for each column
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+-------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance| TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+-------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|   0|    0|         0|        0|  19685|         0|  23500|         0|            0|      

In [52]:
# Mostrar las columnas del DataFrame
print(df.columns)

['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']


In [53]:
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

# Preprocessing
- eliminate unnecesary variables
- missing and duplicates values
- see correlation
- variable transformation
- variable creation

In [5]:
# columns to eliminate
columns = [
    "ArrTime", 
    "ActualElapsedTime", 
    "AirTime", 
    "TaxiIn", 
    "Diverted", 
    "CarrierDelay", 
    "WeatherDelay", 
    "NASDelay", 
    "SecurityDelay", 
    "LateAircraftDelay"
]

# Eliminate columns
df = df.drop(*columns)


In [6]:
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)


In [7]:
# columns to eliminate
columns = [
    "Year",
    "TailNum",
    "TaxiOut",
    "Cancelled",
    "CancellationCode"  
]

# Eliminate columns
df = df.drop(*columns)

In [98]:
df.printSchema()

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)


## Missing values

In [8]:
# Contar valores NA por columna
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|
+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+
|    0|         0|        0|  19685|         0|         0|            0|        0|             0|   23500|   19685|     0|   0|    1015|
+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+


#Now we check if NA stands for 0. If this value is not present, means that NA was 0.

In [16]:
from pyspark.sql import functions as F
# Filter the DataFrame to keep only rows where ArrDelay is equal to 0
filtered_df = df.filter(F.col("ArrDelay") == 0)

# Total number of rows in the DataFrame
total_rows = df.count()

# Check if there are any rows in the filtered DataFrame
if filtered_df.count() > 0:
    print("0 is present in the ArrDelay column " + str(filtered_df.count()) + " times out of " + str(total_rows) + ".")
else:
    print("0 is not present in the ArrDelay column.")

0 is present in the ArrDelay column 60436 times out of 1311826.


In [17]:
# Calculate the percentage of null values for each column
null_percentage = df.select([(count(when(col(c).isNull(), c)) / total_rows).alias(c) for c in df.columns])

# Show the percentage of null values for each column
null_percentage.show()

+-----+----------+---------+--------------------+----------+----------+-------------+---------+--------------+--------------------+--------------------+------+----+--------------------+
|Month|DayofMonth|DayOfWeek|             DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|CRSElapsedTime|            ArrDelay|            DepDelay|Origin|Dest|            Distance|
+-----+----------+---------+--------------------+----------+----------+-------------+---------+--------------+--------------------+--------------------+------+----+--------------------+
|  0.0|       0.0|      0.0|0.015005801074227831|       0.0|       0.0|          0.0|      0.0|           0.0|0.017913961150335486|0.015005801074227831|   0.0| 0.0|7.737306624506603E-4|
+-----+----------+---------+--------------------+----------+----------+-------------+---------+--------------+--------------------+--------------------+------+----+--------------------+


In [20]:
# Drop rows with at least one missing value

df = df.dropna()
dropped_rows = total_rows - df.count()
print("Dropped "+ str(dropped_rows)+ " rows.")

0.018670921295964555
Dropped 24493 rows.


## Duplicates

In [21]:
# Check for duplicates and show the results
total_rows = df.count()
df = df.dropDuplicates()

if total_rows - df.count()  > 0:
    print("There are duplicates in the DataFrame.")
    df.show()
else:
    print("No duplicates found in the DataFrame.")

There are duplicates in the DataFrame.


+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|
+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+
|   10|        10|        6|   1635|      1635|      1732|           PS|     1454|            57|       8|       0|   OAK| BUR|     325|
|   10|        20|        2|   1945|      1945|      2101|           PS|     1484|            76|       1|       0|   SFO| SAN|     447|
|   10|         9|        5|   2057|      2016|      2132|           PS|     1486|            76|      47|      41|   OAK| SAN|     446|
|   10|        13|        2|   2114|      2115|      2223|           PS|     1505|            68|       3|      -1|   SNA| SJC|     342|
|   10|        18|        7|    841|     

## Correlation

## Variable transformation

## Variable creation

# Modeling

# Validation